In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/Vivo_mobile_1.csv")

In [ ]:
df.head()

,Unnamed: 0,Product Name,Offer_Prices,Original_Prices,Description,Reviews
0,0,"vivo Y29 5G (Glacier Blue, 128 GB)","₹15,499","₹17,999",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB16...,4.4
1,1,"vivo T3 Ultra (Frost Green, 256 GB)","₹31,999","₹37,999",8 GB RAM | 256 GB ROM17.22 cm (6.78 inch) Disp...,4.5
2,2,"vivo V40e 5G (Royal Bronze, 128 GB)","₹26,999","₹33,999",8 GB RAM | 128 GB ROM17.2 cm (6.77 inch) Displ...,4.4
3,3,"vivo T2x 5G (Glimmer Black, 128 GB)","₹12,299","₹17,999",4 GB RAM | 128 GB ROM16.71 cm (6.58 inch) Full...,4.4
4,4,"vivo V40e 5G (Mint Green, 256 GB)","₹28,999","₹35,999",8 GB RAM | 256 GB ROM17.2 cm (6.77 inch) Displ...,4.4


In [ ]:
df.columns

Index(['Unnamed: 0', 'Product Name', 'Offer_Prices', 'Original_Prices',
       'Description', 'Reviews'],
      dtype='object')

In [ ]:
# Drop Unnecessary Column if present
if ('Unnamed: 0') in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [ ]:
price_columns = ['Offer_Prices', 'Original_Prices']
for col in price_columns:
    df[col] = df[col].astype(str).str.replace(r'[₹,]', '', regex=True).astype(float)

In [ ]:
# cheack missing vaues
df.isnull().sum()

,0
Product Name,0
Offer_Prices,0
Original_Prices,0
Description,0
Reviews,0


In [ ]:
# Define Target
X = df.drop(columns=['Offer_Prices'])  # Predicting Offer Prices
y = df['Offer_Prices']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

num_features = ['Original_Prices', 'Reviews']
cat_features = ['Product Name']
text_features = ['Description']

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Preprocessing
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
tfidf = TfidfVectorizer()
#TfidfVectorizer() is a text preprocessing tool from sklearn.feature_extraction.\
#text that converts text data into numerical.

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Preprocessing using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('text', TfidfVectorizer(), text_features[0])
    ]
)




In [ ]:
# Create Pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('LinearRegression',LinearRegression())
])


In [ ]:
# Train Initial Model
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
initial_r2 = r2_score(y_test, y_pred)


In [ ]:
print(initial_r2)

0.6690843904915937


In [ ]:
# Fine-Tuning
param_grid = {
    'regressor': [Ridge(), Lasso()],
    'regressor__alpha': [0.01, 0.1, 1, 10, 100]
}


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search = GridSearchCV(Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
]), param_grid, cv=3, scoring='r2')

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.526e+05, tolerance: 5.177e+05
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.713e+05, tolerance: 5.177e+05
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Original_Prices',
                                                                          'Reviews']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['Product '
                                                                          'Name']),
                                                                        ('text',
                                                                         TfidfVectorizer(),
                                                                         'Description')])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor': [Ridge(), Lasso()],
                         'regressor__alpha': [0.01, 0.1, 1, 10, 100]},
             scoring='r2')

In [ ]:
# Best Model After Fine-Tuning
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [ ]:
r2 = r2_score(y_test, y_pred)


In [ ]:
print(r2)

0.9875146479418042


In [ ]:
# user input
print("Enter details for prediction:")
original_price = float(input("Enter Original Price: "))
reviews = float(input("Enter Number of Reviews: "))
product_name = input("Enter Product Name: ")
description = input("Enter Product Description: ")

# again dataframe
user_data = pd.DataFrame([[original_price, reviews, product_name, description]], columns=num_features + cat_features + text_features)

# Future Prices
future_prediction = best_model.predict(user_data)
print("Offer Price:", future_prediction[0])


Enter details for prediction:
Enter Original Price: 40000
Enter Number of Reviews: 4.0
Enter Product Name: vivo
Enter Product Description: 12 gb | 256 gb rom
Offer Price: 25464.829178861342
